<a href="https://colab.research.google.com/github/TurkuNLP/Deep_Learning_in_LangTech_course/blob/master/laser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install laserembeddings
!python -m laserembeddings download-models


✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


In [0]:
import laserembeddings

from laserembeddings import Laser

laser = Laser()

embeddings = laser.embed_sentences(['I love pasta.',"J'adore les pâtes.",'Ich liebe Pasta.'],lang=['en', 'fr', 'de'])

print(embeddings)
print(embeddings.shape)

#can this be any simpler? :)


[[-5.2039017e-04 -2.8321840e-05 -1.6871469e-04 ...  3.4788840e-03
  -1.9968930e-03  8.1148231e-03]
 [ 3.2193204e-03 -9.9815654e-05  5.9067555e-05 ...  7.6490263e-03
   1.1962679e-03  2.4502634e-03]
 [ 5.3412444e-04 -3.6210116e-05 -1.4794576e-04 ...  6.1386470e-03
  -1.6569846e-03  6.3126544e-03]]
(3, 1024)


In [0]:
!wget http://dl.turkunlp.org/.ginter/hs.txt
!wget http://dl.turkunlp.org/.ginter/yle.txt

--2020-04-22 14:07:38--  http://dl.turkunlp.org/.ginter/hs.txt
Resolving dl.turkunlp.org (dl.turkunlp.org)... 195.148.30.23
Connecting to dl.turkunlp.org (dl.turkunlp.org)|195.148.30.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24291 (24K) [text/plain]
Saving to: ‘hs.txt.5’

hs.txt.5            100%[===================>]  23.72K  69.9KB/s    in 0.3s    

2020-04-22 14:07:39 (69.9 KB/s) - ‘hs.txt.5’ saved [24291/24291]

--2020-04-22 14:07:42--  http://dl.turkunlp.org/.ginter/yle.txt
Resolving dl.turkunlp.org (dl.turkunlp.org)... 195.148.30.23
Connecting to dl.turkunlp.org (dl.turkunlp.org)|195.148.30.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21392 (21K) [text/plain]
Saving to: ‘yle.txt.5’

yle.txt.5           100%[===================>]  20.89K  65.7KB/s    in 0.3s    

2020-04-22 14:07:43 (65.7 KB/s) - ‘yle.txt.5’ saved [21392/21392]



* The two files `yle.txt` and `hs.txt` contain some 200+ news titles from YLE and HS, judged by a human to be paraphrases or near-paraphrases of each other
* The selection is such that lexical overlap is minimized
* We could make a simple test of LASER, comparing them against each other to see if we can pair these up

In [0]:
def read_file(fname):
  lines=[]
  with open(fname) as f:
    for line in f:
      line=line.strip()
      if not line:
        continue
      lines.append(line)
  return lines

hs=read_file("hs.txt")
yle=read_file("yle.txt")

hs_vectors=laser.embed_sentences(hs,"fi")
yle_vectors=laser.embed_sentences(yle,"fi")

print("hs",hs_vectors.shape)
print("yle",yle_vectors.shape)

hs (217, 1024)
yle (217, 1024)


In [0]:
import sklearn.metrics
all_dist=sklearn.metrics.pairwise_distances(hs_vectors,yle_vectors)
print(all_dist.shape)
#we get a document-by-document matrix, with distances

(217, 217)


In [0]:
nearest=all_dist.argmin(axis=-1) #These are the nearest neighbors for each HS title (indices into YLE), perfect solution would be [0,1,2,3...,216]
print(nearest)

[121   1   1   3 120 116   6   9   8   9 177  11 102  13  44  15 126  17
 196 206 189  96  22  23  70   9  77 189 216  67  30 136  78  61  34  74
  53  37 161  39  40 147 203  13  44  45   9  47  48  49  14  51 120  53
  54  55 121  46  58 123  60  61 189  48  64  65  39  13 126  69  70  71
  61  73  74  61  76  26 189  74  14  81  82  83  84  85  86  87 182  64
  15 135 176 189  70  59 206  97 208  99  99  84 102 103 104 105  48  67
  91 175 110  14 112 113  39 115 116 121 118  73 120 121 121 123 124 125
 126 159 128 129 130  92 132 133 134  61 136 137 138   9  39 141 142 123
  39   9  35  15 150 149  67 151 152 153 180 155 156 137 158  79 160 176
  79 163 164 112  39 167 168 169 132 171 172 146  61 175 201 177 178 179
 180  63 189 183 184  67  39 187 197 189  94 146 192 193 150 195 196 197
  78  39 198  74 202 203 204 133 206   9 208 146  86  59  30 213 214 215
 216]


In [0]:
def eval_embeddings(hs,yle,hs_vectors,yle_vectors):
  all_dist=sklearn.metrics.pairwise_distances(hs_vectors,yle_vectors)
  nearest=all_dist.argmin(axis=-1) #These are the nearest neighbors for each HS title (indices into YLE), perfect solution would be [0,1,2,3...,216]   
  correct=0
  for i,hs_txt in enumerate(hs):
    j=nearest[i]
    if i==j:
      correct+=1
    yle_txt=yle[j]
    print(i==j)
    print(hs_txt)
    print(yle_txt)
    print()

  print("CORRECT",correct,"/",len(hs),"=",correct/len(hs)*100,"%")

In [0]:
eval_embeddings(hs,yle,hs_vectors,yle_vectors)

False
Mansikkahuijauksia aletaan suitsia kemian keinoin: Isotooppi­analyysi paljastaa marjan alkuperän
Poliisilla iso urakka edessä: Supo auttaa KRP:tä Turun saaristosta löytyneen materiaalin läpikäymisessä

True
Tutkimus: Suomalaiset kuuluvat maailman tiede­myönteisimpiin kansoihin
Suomalaisten luotto tieteeseen on vankka

False
Kuluttajien luottamus Suomen talouteen heikkeni
Suomalaisten luotto tieteeseen on vankka

True
Ensimmäinen avaruuteen lähetetty suomalais­satelliitti tuhoutui tähden­lentona
Aalto-2 paloi poroksi ilmakehässä

False
Kysely: Valtaosa suomalaisista ei halua suurta perintöä vanhemmiltaan
Reilu puolet suomalaisista vastustaa Natoon liittymistä

False
Muuttovalmiit valmistalot ohittivat suosiossa perinteiset talopaketit
Matkailijat tuovat nyt mietoja juomia väkevien suosion hiipuessa

True
Kansantaudit tappavat suomalaisia aiempaa harvemmin, mutta edessä siintää uusi uhka: lihavuus
Suomalaisten riski kuolla ennenaikaisesti pienentynyt, lihominen hyvän kehityksen jar

# Try with BERT?

*   We could try with BERT
*   Test the [CLS] token as the sentence embedding
*   Test the average of token embeddings as the sentence embedding



In [0]:
#Note: since LASER is torch, maybe we continue in torch for the fun of it? :)
!pip install transformers
import transformers
bert_model = transformers.BertModel.from_pretrained("bert-base-finnish-cased-v1") 
bert_model = model.cuda() #move the model to GPU
bert_model.eval() #tell the model it will be used for predictions, not training


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(50105, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [0]:
import torch
import torch.nn

#Load the Finnish BERT tokenizer
tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-finnish-cased-v1")

def tokenize_texts(texts):
  tokenized_ids=[tokenizer.encode(txt,add_special_tokens=True) for txt in texts] #this runs the BERT tokenizer, returns list of lists of integers
  tokenized_ids_t=[torch.tensor(ids,dtype=torch.long) for ids in tokenized_ids] #turn lists of integers into torch tensors
  tokenized_single_batch=torch.nn.utils.rnn.pad_sequence(tokenized_ids_t,batch_first=True) #zero-padding
  return tokenized_single_batch

hs_data=tokenize_texts(hs)
yle_data=tokenize_texts(yle)

print(hs_data.shape)

torch.Size([217, 37])


In [0]:
data=hs_data
with torch.no_grad(): #tell the model not to gather gradients
  emb=model(data.cuda()) #applies the model and returns several things, we care about the first
  print(emb[0].shape)  # word x sequence x embedding


torch.Size([217, 37, 768])


In [0]:
def embed(data):
  with torch.no_grad(): #tell the model not to gather gradients
    emb=bert_model(data.cuda()) #runs BERT and returns several things, we care about the first
    #emb[0]  # batch x word x embedding
    pooled=emb[0].mean(1) #average along the word dimension -> pools the embeddings into one
    #pooled=emb[0][:,0,:].squeeze() #Pick the first token as the embedding
    print(pooled.shape)
  return pooled.cpu().numpy() #done! move data back to CPU and extract the numpy array

hs_emb=embed(hs_data)
yle_emb=embed(yle_data)


torch.Size([217, 768])
torch.Size([217, 768])


In [0]:
eval_embeddings(hs,yle,hs_emb,yle_emb)

False
Mansikkahuijauksia aletaan suitsia kemian keinoin: Isotooppi­analyysi paljastaa marjan alkuperän
Tutkinta alkoi 40 euron velasta: Poliisi paljasti nuorten huumeringin Pirkanmaalla

False
Tutkimus: Suomalaiset kuuluvat maailman tiede­myönteisimpiin kansoihin
Reilu puolet suomalaisista vastustaa Natoon liittymistä

False
Kuluttajien luottamus Suomen talouteen heikkeni
Eduskunta aloittaa kesälomansa

False
Ensimmäinen avaruuteen lähetetty suomalais­satelliitti tuhoutui tähden­lentona
Poliisi epäilee: Kaksi miestä kuoli salaman iskuun Juuassa

False
Kysely: Valtaosa suomalaisista ei halua suurta perintöä vanhemmiltaan
Aseluvan verkkohaku lykkääntyy jo toistamiseen

False
Muuttovalmiit valmistalot ohittivat suosiossa perinteiset talopaketit
Etlan tutkimus: Työttömyys pistää muuttokuorman herkästi liikkeeseen

False
Kansantaudit tappavat suomalaisia aiempaa harvemmin, mutta edessä siintää uusi uhka: lihavuus
Suomi tuskailee ylipainon kanssa – WHO:n tavoitteiden saavuttaminen näyttää va

BERT [CLS] embedding is the worst, average of BERT embeddings comes then, and LASER mops the floor with these
